In [1]:
import Gadget as G
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from matplotlib.mlab import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition, mark_inset)
import numpy            as np
import math             as mt
import statistics       as st
import snap_blocks_crun as sbc
import plots            as im
import plot_format      as pf
import global_fuctions  as gf
import mpl_toolkits

/usr/lib64/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
# parameters 
v0      = 6.23e7
u0      = v0*v0
kB      = 1.38e-16
mp      = 1.67e-24
xNe     = 1.
nHe_fak = 3.
xH      = 0.76
yHe     = (1-xH)/(4*xH)
mu      = (1+4*yHe)/(1+nHe_fak*yHe+xNe)
T0      = mp*u0*mu/kB
rho0    = 4.03e-12
m0      = 6.97e39

In [3]:
def save_file(path,var,name,time):
    sfile = open(path+name,'a')
    sfile.write('{time:5d} {media:15.5e} {mediana:15.5e} {sigma:15.5e} \n'
          .format(time=time,media=var[0],mediana=var[1],sigma=var[2]))
    sfile.close()
    
def save_file2(path,var,name,time):
    sfile = open(path+name,'a')
    sfile.write('{time:5d} {media:15.5e} {mediana:15.5e} {sigma:15.5e} {per25:15.5e} {per75:15.5e} \n'
          .format(time=time,media=var[0],mediana=var[1],sigma=var[2],per25=var[3],per75=var[4]))
    sfile.close()    

In [4]:
#declaration of paths
path_main = '/home/meugenia/Documentos/discos/'
path_snap = path_main+'snaps/controlrun/'
path_save = path_main +'plots/'
dir_txt   = 'regiones/disco/propiedades_vs_t/unidades_correctas/control/'
##snap = 'snapshot_000'
#ids, x,y,z,mass,rho,vx,vy,vz,bx,by,bz,vrms,hsml,eint = sb.reading_snap(path_snap,snap,0)
#ids_s,x_s,y_s,z_s,mass_s,rho_s,vx_s,vy_s,vz_s,bx_s,by_s,bz_s,vrms_s,hsml_s,eint_s = \
#sb.sorting_by_ids(ids,x,y,z,mass,rho,vx,vy,vz,bx,by,bz,vrms,hsml,eint)
#r=(x_s*x_s+y_s*y_s)**0.5

In [5]:
sindex=["000", 
        "010","020","030","040","050","060","070","080","090","100",
        "110","120","130","140","150","160","170","180","190","200",
        "210","220","230","240","250","260","270","280","290","300",
        "310","320","330","340","350","370"]

In [10]:
for sn in range(len(sindex)):
    #if sn!=0:
    #    continue
    snap = 'snapshot_'+sindex[sn]
    print snap
    ids, x,y,z,mass,rho,vx,vy,vz,vrms,hsml,eint = sbc.reading_snap(path_snap,snap,0)
    ids_s,x_s,y_s,z_s,mass_s,rho_s,vx_s,vy_s,vz_s,vrms_s,hsml_s,eint_s = \
    sbc.sorting_by_ids(ids,x,y,z,mass,rho,vx,vy,vz,vrms,hsml,eint)
    r=(x_s*x_s+y_s*y_s)**0.5
    
    limx    = [-10,10]
    limy    = [-10,10]
    #limz    = [-10,10]    
    limz    = [-1,1]    
    limrho  = [0.0001,np.max(rho_s)]
    limeint = [0,0.015]
    limvrms = [-3,0]
    #limr    = [0,10]
    limr    = [2,7]    
    
    sgvar = [x_s, y_s, z_s, r, rho_s, mass_s, vx_s, vy_s, vz_s, vrms_s, eint_s]
    
    #var   = [4,10]
    #vlims = [limrho,limeint]
    var   = [3,2]
    vlims = [limr,limz]
    
    for i in range(len(var)):
        #print vlims[i]
        if i==0:
            _sgvar=gf.sobgroup(sgvar,var[i],vlims[i])
            sgvar=_sgvar
            print var[i], _sgvar[var[i]][0].min(), _sgvar[var[i]][0].max()
        else:
            _sgvar=gf.sobgroup([sgvar[0][0],sgvar[1][0],sgvar[2][0],sgvar[3][0],sgvar[4][0],\
                               sgvar[5][0],sgvar[6][0],sgvar[7][0],sgvar[8][0],sgvar[9][0],\
                               sgvar[10][0]], var[i],vlims[i])
            sgvar=_sgvar
            #print var[i], _sgvar[var[i]][0].min(), _sgvar[var[i]][0].max()
            
            
    sgx,sgy,sgz,sgr,sgrho,sgmass,sgvx,sgvy,sgvz,sgvrms,sgeint = gf.subgroup_all_crun(_sgvar)  
    
    
    #sgbr   =  sgbx*np.cos(np.arctan2(sgy,sgx)) + sgby*np.sin(np.arctan2(sgy,sgx))
    #sgbphi = -sgbx*np.sin(np.arctan2(sgy,sgx)) + sgby*np.cos(np.arctan2(sgy,sgx))
    
    #sgemagr   = 0.125*(sgbr**2   )*sgmass/sgrho
    #sgemagp   = 0.125*(sgbphi**2 )*sgmass/sgrho
    #sgemagz   = 0.125*(sgbz**2   )*sgmass/sgrho
    
    #bmodc     = sgbx**2+sgby**2+sgbz**2
    
    # correct units
    rho_   = sgrho*rho0
    ecin_  = 0.5*sgmass*v0**2*(sgvx**2+sgy**2+sgz**2)
    T_     = (5/float(3)-1)*sgeint*T0
    eint_  = sgeint*sgmass*m0*u0
    #emag_  = 0.125/np.pi*bmodc*sgmass*m0/rho_
    etot_  = ecin_+eint_
        
    mediarho   ,medianarho   ,sigmarho   ,percentil25rho   ,percentil75rho   = st.internal_stat(rho_)
    mediaeint  ,medianaeint  ,sigmaeint  ,percentil25eint  ,percentil75eint  = st.internal_stat(eint_)
    mediavrms  ,medianavrms  ,sigmavrms  ,percentil25vrms  ,percentil75vrms  = st.internal_stat(sgvrms)
    #mediabx    ,medianabx    ,sigmabx    ,percentil25bx    ,percentil75bx    = st.internal_stat(sgbx)
    #mediaby    ,medianaby    ,sigmaby    ,percentil25by    ,percentil75by    = st.internal_stat(sgby)
    #mediabz    ,medianabz    ,sigmabz    ,percentil25bz    ,percentil75bz    = st.internal_stat(sgbz)
    #mediabr    ,medianabr    ,sigmabr    ,percentil25br    ,percentil75br    = st.internal_stat(sgbr)
    #mediabphi  ,medianabphi  ,sigmabphi  ,percentil25bphi  ,percentil75bphi  = st.internal_stat(sgbphi)
    #mediabzc   ,medianabzc   ,sigmabzc   ,percentil25bzc   ,percentil75bzc   = st.internal_stat(sgbz**2)
    #mediabrc   ,medianabrc   ,sigmabrc   ,percentil25brc   ,percentil75brc   = st.internal_stat(sgbr**2)
    #mediabphic ,medianabphic ,sigmabphic ,percentil25bphic ,percentil75bphic = st.internal_stat(sgbphi**2)
    #mediabc    ,medianabc    ,sigmabc    ,percentil25bc    ,percentil75bc    = st.internal_stat(sgbz**2+sgbphi**2*sgbr**2)
    #mediaemagz ,medianaemagz ,sigmaemagz ,percentil25emagz ,percentil75emagz = st.internal_stat(sgemagz)
    #mediaemagr ,medianaemagr ,sigmaemagr ,percentil25emagr ,percentil75emagr = st.internal_stat(sgemagr)
    #mediaemagp ,medianaemagp ,sigmaemagp ,percentil25emagp ,percentil75emagp = st.internal_stat(sgemagp)
        
    mediaecin ,medianaecin ,sigmaecin ,percentil25ecin ,percentil75ecin = st.internal_stat(ecin_)
    mediaT    ,medianaT    ,sigmaT    ,percentil25T    ,percentil75T    = st.internal_stat(T_)
    #mediaemag ,medianaemag ,sigmaemag ,percentil25emag ,percentil75emag = st.internal_stat(emag_)
    mediaetot ,medianaetot ,sigmaetot ,percentil25etot ,percentil75etot = st.internal_stat(etot_)
        
    times=snap[9::]
    timei=int(times)
    print 'time: ', timei
    
    save_file2(path_main+dir_txt,[mediarho   ,  medianarho   ,  sigmarho  ,percentil25rho   ,percentil75rho  ],'mc_evo_rho'  ,timei)
    save_file2(path_main+dir_txt,[mediaeint  ,  medianaeint  ,  sigmaeint ,percentil25eint  ,percentil75eint ],'mc_evo_eint' ,timei)
    save_file2(path_main+dir_txt,[mediavrms  ,  medianavrms  ,  sigmavrms ,percentil25vrms  ,percentil75vrms ],'mc_evo_vrms' ,timei)
    #save_file2(path_main+dir_txt,[mediabx    ,  medianabx    ,  sigmabx   ,percentil25bx    ,percentil75bx   ],'mef_evo_bx'   ,timei)
    #save_file2(path_main+dir_txt,[mediaby    ,  medianaby    ,  sigmaby   ,percentil25by    ,percentil75by   ],'mef_evo_by'   ,timei)
    #save_file2(path_main+dir_txt,[mediabz    ,  medianabz    ,  sigmabz   ,percentil25bz    ,percentil75bz   ],'mef_evo_bz'   ,timei)
    #save_file2(path_main+dir_txt,[mediabr    ,  medianabr    ,  sigmabr   ,percentil25br    ,percentil75br   ],'mef_evo_br'   ,timei)
    #save_file2(path_main+dir_txt,[mediabphi  ,  medianabphi  ,  sigmabphi ,percentil25bphi  ,percentil75bphi ],'mef_evo_bphi' ,timei)
    #save_file2(path_main+dir_txt,[mediabzc   ,  medianabzc   ,  sigmabzc  ,percentil25bzc   ,percentil75bzc  ],'mef_evo_bzc'  ,timei)
    #save_file2(path_main+dir_txt,[mediabrc   ,  medianabrc   ,  sigmabrc  ,percentil25brc   ,percentil75brc  ],'mef_evo_brc'  ,timei)
    #save_file2(path_main+dir_txt,[mediabphic ,  medianabphic ,  sigmabphic,percentil25bphic ,percentil75bphic],'mef_evo_bphic',timei)
    #save_file2(path_main+dir_txt,[mediabc    ,  medianabc    ,  sigmabc   ,percentil25bc    ,percentil75bc   ],'mef_evo_bc'   ,timei)
    #save_file2(path_main+dir_txt,[mediaemagz ,  medianaemagz ,  sigmaemagz,percentil25emagz ,percentil75emagz],'mef_evo_emagz',timei)
    #save_file2(path_main+dir_txt,[mediaemagr ,  medianaemagr ,  sigmaemagr,percentil25emagr ,percentil75emagr],'mef_evo_emagr',timei)
    #save_file2(path_main+dir_txt,[mediaemagp ,  medianaemagp ,  sigmaemagp,percentil25emagp ,percentil75emagp],'mef_evo_emagp',timei)    
    
    save_file2(path_main+dir_txt,[mediaecin  ,  medianaecin  ,  sigmaecin ,percentil25ecin  ,percentil75ecin] ,'mc_evo_ecin',timei)
    save_file2(path_main+dir_txt,[mediaT     ,  medianaT     ,  sigmaT    ,percentil25T     ,percentil75T]    ,'mc_evo_T'   ,timei)
    #save_file2(path_main+dir_txt,[mediaemag  ,  medianaemag  ,  sigmaemag ,percentil25emag  ,percentil75emag] ,'mef_evo_emag',timei)
    save_file2(path_main+dir_txt,[mediaetot  ,  medianaetot  ,  sigmaetot ,percentil25etot  ,percentil75etot] ,'mc_evo_etot',timei)

    
   

snapshot_000
3 2.0000005 6.9999914
time:  0
snapshot_010
3 2.0000007 6.999979
time:  10
snapshot_020
3 2.0000012 6.9999895
time:  20
snapshot_030
3 2.0000014 6.999995
time:  30
snapshot_040
3 2.000006 6.999965
time:  40
snapshot_050
3 2.0000002 6.999985
time:  50
snapshot_060
3 2.0000026 6.9999824
time:  60
snapshot_070
3 2.000004 6.9999914
time:  70
snapshot_080
3 2.0000007 6.9999843
time:  80
snapshot_090
3 2.0000126 6.999951
time:  90
snapshot_100
3 2.000008 6.9999847
time:  100
snapshot_110
3 2.0000057 6.99999
time:  110
snapshot_120
3 2.0000029 6.9999905
time:  120
snapshot_130
3 2.0000114 6.9999757
time:  130
snapshot_140
3 2.0000007 6.999997
time:  140
snapshot_150
3 2.0000112 6.9999986
time:  150
snapshot_160
3 2.000007 6.999974
time:  160
snapshot_170
3 2.0000026 6.999997
time:  170
snapshot_180
3 2.0000002 6.999985
time:  180
snapshot_190
3 2.0000024 6.9999886
time:  190
snapshot_200
3 2.0000002 6.999996
time:  200
snapshot_210
3 2.000007 6.9999495
time:  210
snapshot_220
3 2

UnboundLocalError: local variable 'curfilename' referenced before assignment